In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.feature_selection import VarianceThreshold

In [8]:
data = pd.read_csv('new_dataset/owid-covid-data-most-recent.csv')
print(data.shape)
data = data.dropna(subset = ['new_deaths'])
print(data.shape)

# converting date to date object
data['date'] = pd.to_datetime(data['date'])

# dropping observations before COVID-19 was detected in the country
data = data[~data['total_cases'].isna()]

# dropping redundant columns like smoothed and similar predictors
# maybe someone else go through this and check just in case
columns_to_drop = ['new_cases_smoothed',
                   'new_cases_smoothed_per_million',
                   'total_deaths',
                   'total_deaths_per_million',
                   'new_deaths_per_million',
                   'new_deaths_smoothed',
                   'new_deaths_smoothed_per_million',
                   'excess_mortality_cumulative',
                   'excess_mortality_cumulative_absolute',
                   'excess_mortality_cumulative_per_million',
                   'icu_patients',
                   'hosp_patients',
                   'weekly_icu_admissions',
                   'weekly_hosp_admissions',
                   'total_tests',
                   'new_tests',
                   'new_tests_smoothed',
                   'new_tests_smoothed_per_thousand',
                   'tests_per_case', #this is the inverse of positive_rate
                   'new_vaccinations_smoothed',
                   'total_vaccinations',
                   'people_vaccinated',
                   'people_fully_vaccinated',
                   'total_boosters',
                   'new_vaccinations_smoothed_per_million',
                   'new_people_vaccinated_smoothed']
data = data.drop(columns_to_drop,axis=1)

# filtering countries by null values in other columns
null_counts = data.groupby('location').agg(lambda x: x.isnull().sum(axis=0))
# missing_counts = data.groupby('location').apply(lambda x: x.isnull().sum())
total_null = null_counts.sum(axis=1)
null_counts['total_null'] = null_counts.sum(axis=1)
#filtering top 10 countries 
sorted_countries = null_counts[['total_null']].sort_values(by = 'total_null',ascending=True)
top_df = sorted_countries.iloc[:10]
filtered_countries_list = top_df.index.to_list()
data = data[data['location'].isin(filtered_countries_list)]
print(data.shape)
total_missing_per_country = total_null
sorted_countries = total_missing_per_country.sort_values(ascending=True)
sorted_countries.head(10)

(276420, 67)
(51621, 67)
['United States', 'Estonia', 'Italy', 'Ireland', 'Israel', 'Czechia', 'Malaysia', 'Belgium', 'Chile', 'France']


(2007, 41)

In [10]:
missingness_prop = sorted_countries / len(data)
sorted_countries = sorted_countries.to_frame(name='total_missing').join(missingness_prop.rename('missingness_prop'))
sorted_countries.head(10)

,total_missing,missingness_prop
location,,
United States,1008,0.502242
Estonia,1104,0.550075
Italy,1191,0.593423
Ireland,1285,0.640259
Israel,1367,0.681116
Czechia,1408,0.701545
Malaysia,1465,0.729945
Belgium,1514,0.754360
Chile,1559,0.776781


In [20]:
#missingness by variable
def highlight(val):
    color = 'red' if val > 0.5 else 'black'
    return f'color: {color}'

missing_values = data.isnull().sum()
missingness_prop = missing_values / len(data)
column_missingness = missing_values.to_frame(name='total_missing').join(missingness_prop.rename('missingness_prop'))
column_missingness = column_missingness.style.applymap(highlight, subset=['missingness_prop'])
column_missingness

,total_missing,missingness_prop
iso_code,0,0.000000
continent,0,0.000000
location,0,0.000000
date,0,0.000000
total_cases,0,0.000000
new_cases,0,0.000000
new_deaths,0,0.000000
total_cases_per_million,0,0.000000
new_cases_per_million,0,0.000000
reproduction_rate,532,0.265072


In [22]:
#correlation matrix
corr = pd.DataFrame(data.corrwith(data["new_deaths"]))
corr = corr.rename(columns={0: 'Correlation with new_deaths'})
corr.style.applymap(highlight)

/var/folders/8k/n_ksdn3d24l74_zsdlmr7drh0000gn/T/ipykernel_27451/2470314427.py:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = pd.DataFrame(data.corrwith(data["new_deaths"]))


,Correlation with new_deaths
total_cases,0.350927
new_cases,0.642166
new_deaths,1.000000
total_cases_per_million,-0.168407
new_cases_per_million,0.092862
reproduction_rate,-0.104465
icu_patients_per_million,0.338267
hosp_patients_per_million,0.287799
weekly_icu_admissions_per_million,0.461244
weekly_hosp_admissions_per_million,0.305290


In [ ]:
num_col = data.select_dtypes(include=['number']).columns
data_num = data[num_col]
threshold = 0.95
selector = VarianceThreshold(threshold)
selector.fit(data_num)
quasi_constant_indices = data_num.columns[~selector.get_support()]
quasi_constant_indices

In [ ]:
# Variables to remove:
#       quasi-constant variables: 'reproduction_rate', 'positive_rate', 'extreme_poverty','handwashing_facilities', 'human_development_index']
#       low correlation and high missigness: 'total_boosters_per_hundred'
#       handwashing_facilities is 100% missing in the smaller data set